# Лабораторная работа 6

## Выравнивание списка, состоящего из итерируемых объектов, на основе рекурсии

*Ляпунова Арина Сегреевна, 30.04.2025* 

In [49]:
import sys
sys.getrecursionlimit()

3000

## Задание 6.1. Выравнивание вложенных списков

***Задание 6.1.1. Рекурсивная функция flatten_list_v1***

In [50]:
it_list = [[1],[2,[3]],[[[[[4]]]]],5,6,7]

In [51]:
def flatten_list_v1(nested_obj):
    result = []  # Инициализируем пустой список для хранения результата
    if isinstance(nested_obj,list): #  является ли объект списком
        for item in nested_obj: # проходимся по элементам списка
            result += flatten_list_v1(item) # рекурсия
    else:            #Если это не список, добавляем элемент в итоговый список как есть
        result += [nested_obj] # добавляем в результат
    return result

In [52]:
flatten_list_v1(it_list)

[1, 2, 3, 4, 5, 6, 7]

***Задание 6.1.2. Генераторная функция на основе рекурсии flatten_list_v2***

На основе функции `flatten_list_v1` напишите генераторную функцию `flatten_list_v2(nested_obj)`, которая выравнивает вложенный список и возвращает генератнорый объект для элементов вложенного списка.

Напишите комментарии для каждой строки кода функции `flatten_list_v2`.

Предложите два варианта реализации: только с использованием `yield (вариант 1)` и с использованием `yield from(вариант 2)`.

*1 Вариант:*

In [107]:
def flatten_list_v2(nested_obj):
    if isinstance(nested_obj, list): # Проверяем, является ли текущий объект списком
        for item in nested_obj:# Если это список, перебираем его элементы
            for element in flatten_list_v2(item):
                yield element
    else:
        yield nested_obj

In [108]:
print([x for x in flatten_list_v2(it_list)])

[1, 2, 3, 4, 5, 6, 7]


*2 Вариант*:

In [109]:
def flatten_list_v2(nested_obj):
    if isinstance(nested_obj, list): # Проверяем, является ли текущий объект списком
        for item in nested_obj:  # Если это список, перебираем его элементы
            yield from flatten_list_v2(item) 
    else:
        yield nested_obj

In [56]:
print([x for x in flatten_list_v2(it_list)])

[1, 2, 3, 4, 5, 6, 7]


***Задание 6.1.3. Функция flatten_list: создание, документирование, тестирование***

Объедините две пользовательские функции `flatten_list_v1` и `flatten_list_v2` в одну пользовательскую функцию `flatten_list(nested_obj, gen)`, которая возвращает либо список из элементов вложенных списков без сохранения структуры, если `gen=False`, либо возвращает генераторный объект, если `gen=True`. Стандартным значением для аргумента `gen` является объект `False`.

In [57]:
def flatten_list(nested_obj, gen=False):
    """ Возвращает список или генераторный объект элементов вложенных списков """
    if gen:
        return flatten_list_v1(nested_obj)  
    else:
        return flatten_list_v2(nested_obj)  

In [58]:
[x for x in flatten_list_v2(it_list)]

[1, 2, 3, 4, 5, 6, 7]

Три примера вызова функции flatten_list для различных значений аргументов:

In [59]:
my_list = [1, [2, [[3, 4]], 5]]#Функция вызвана без указания gen, поэтому возвращает плоский список всех элементов, рекурсивно обойдя вложенные структуры.
result = flatten_list(my_list)
print(result)

<generator object flatten_list_v2 at 0x0000016E062998C0>


In [60]:
complex_list = [1, ["a", [2.5, ("hello",)], True], {"key": "value"}]
result = flatten_list(complex_list, gen=False)
print(result)

<generator object flatten_list_v2 at 0x0000016E0629BBC0>


In [63]:
large_list = [[n, [n*2, [n*3]]] for n in range(1,100,4)]
generator = flatten_list(large_list, gen=False)
some_elements = [next(generator) for _ in range(7)]
print(some_elements)

[1, 2, 3, 5, 10, 15, 9]


## Задание 6.2. Выравнивание вложенныхитерируемых объектов произвольного типа

In [32]:
it_list

[[1], [2, [3]], [[[[[4]]]]], 5, 6, 7]

In [33]:
it_tuple = (((7),(8)),(9),10)

In [34]:
it_str = "abcdefgh"

In [35]:
it_dict = {"key1": 11, "key2": {"key3": 12}}

In [36]:
it_gen = ([i,i**2,i**3] for i in range(5))

In [37]:
it_file = open("flatten_file.txt")

Объединим все итерируемые объекты в список

In [38]:
it = [it_list, it_tuple, it_str, it_dict, it_gen, it_file]

Все итерируемые объекты имеют метод __iter__ для создания объекта итератора

In [39]:
['__iter__' in dir(x) for x in it]

[True, True, True, True, True, True]

In [40]:
[hasattr(x, '__iter__') for x in it]

[True, True, True, True, True, True]

***Задание 6.2.1. Рекурсивная функция flatten_it_v1***

In [41]:
def flatten_it_v1(nested_obj):
    result = [] 
    if hasattr(nested_obj, '__iter__'):  # проверяем, является ли объект итерируемым
        if isinstance(nested_obj, str):  # если строка, не итерируем
            result += [nested_obj]  # добавляем строку
        elif isinstance(nested_obj, dict):  # если словарь, итерируем по ключам и значениям
            for key, value in nested_obj.items():
                result += flatten_it_v1(key)  # добавляем ключи
                result += flatten_it_v1(value)  # добавляем значения
        else:
            for item in nested_obj:  # проходим по элементам итерируемого объекта
                result += flatten_it_v1(item)  # рекурсивно добавляем элементы
    else:
        result += [nested_obj]  # добавляем элемент, если он не итерируем
    return result  

In [42]:
print(flatten_it_v1(it_list))
print(flatten_it_v1(it_dict))

[1, 2, 3, 4, 5, 6, 7]
['key1', 11, 'key2', 'key3', 12]


***Задание 6.2.2. Генераторная функция на основе рекурсии flatten_it_v2***

На основе функций `flatten_it_v`1 и `flatten_list_v2` напишите пользовательскую функцию `flatten_it_v2(nested_obj)` , которая выравнивает вложенную структуру из различных итерируемых объектов и возвращает генератнорый объект элементов без сохранения структуры

In [64]:
def flatten_it_v2(nested_obj):
    if isinstance(nested_obj, str):  # Сначала проверяем строки отдельно
        yield nested_obj
    elif hasattr(nested_obj, '__iter__'):  # Затем всё остальное итерируемое
        if isinstance(nested_obj, dict):
            for key, value in nested_obj.items():
                yield from flatten_it_v2(key)
                yield from flatten_it_v2(value)
        else:
            for item in nested_obj:
                yield from flatten_it_v2(item)
    else:  # Простые неитерируемые объекты
        yield nested_obj

In [65]:
print([x for x in flatten_it_v2(it_list)])

[1, 2, 3, 4, 5, 6, 7]


***Задание 6.2.3. Функция flatten_it: создание, документирование, тестирование***

In [66]:
def flatten_it(nested_obj, gen=False):
    def _flatten_gen(obj):
        if isinstance(obj, str):  # Сначала проверяем строки отдельно
            yield obj
        elif hasattr(obj, '__iter__'):  # Затем всё остальное итерируемое
            if isinstance(obj, dict):
                for key, value in obj.items():
                    yield from _flatten_gen(key)
                    yield from _flatten_gen(value)
            else:
                for item in obj:
                    yield from _flatten_gen(item)
        else:  # Простые неитерируемые объекты
            yield obj
    
    return _flatten_gen(nested_obj) if gen else list(_flatten_gen(nested_obj))

In [67]:
data = [1, {'x': (2, [3])}, "text"]
print(flatten_it(data)) 

[1, 'x', 2, 3, 'text']


In [68]:
print(flatten_it("hello", gen=False))  

['hello']


In [69]:
data = [1, {"a": ("hello", [2.5, {3, 4}])}, [5, {"b": None}]]
result = flatten_it(data)
print(result)

[1, 'a', 'hello', 2.5, 3, 4, 5, 'b', None]


## Задание 6.3. Обработка циклических объектов

In [70]:
it_list_cyclic = it_list[:]
it_list_cyclic.append(it_list_cyclic)

In [71]:
it_list_cyclic

[[1], [2, [3]], [[[[[4]]]]], 5, 6, 7, [...]]

In [72]:
def flatten_list_v1(nested_obj):
    result = []
    if isinstance(nested_obj,list):
        for item in nested_obj:
            if item is nested_obj:
                raise ValueError("Cyclic list is found") # генерация исключения
            else:
                result += flatten_list_v1(item)
    else:
        result += [nested_obj]
    return result

In [73]:
flatten_list_v1(it_list)

[1, 2, 3, 4, 5, 6, 7]

In [74]:
flatten_list_v1(it_list_cyclic)

ValueError: Cyclic list is found

Для того, чтобы стандартное сообщение об ошибке не выводилось, перехватим
созданное нами исключение `ValueError` внутри функции `flatten_list_v1` с
использованием оператора `try` в сочетании с `except` . Если внутри блока `try`
возникнет исключение `ValueError` , то процесс выполнения кода перейдет в блок
оператора `except` , для которого указано исключение `ValueError`

In [75]:
try:
 print(flatten_list_v1(it_list_cyclic))
except ValueError as e:
 print(e)


Cyclic list is found


***Задание 6.3.1. Доопределение функции flatten_it***

Доопределите пользовательскую функцию `flatten_it` 

In [103]:
def flatten_it(nested_obj, gen=False):
    
    visited = set()  # Для хранения id посещенных объектов

    def flatten_gen(obj):
        obj_id = id(obj) # Получаем уникальный идентификатор объекта, чтоюы отслеживать повторяющтеся объекты
        if isinstance(obj, (list, dict, set)):# Проверяем только изменяемые итерируемые объекты
            if obj_id in visited:
                raise ValueError("Ахтунг, обнаружен цикл")
            visited.add(obj_id)
            try:
                if isinstance(obj, dict): # Обрабатываем словарь как в предыдущих функциях 
                    for key, value in obj.items():
                        yield from flatten_gen(key)
                        yield from flatten_gen(value)
                else: # Обрабатываем списки/множества/кортежи
                    for element in obj:
                        yield from flatten_gen(element)
            finally:
                visited.remove(obj_id)  # Без finally объект мог бы остаться в множестве visited, если при обработке его элементов возникнет исключение
        elif isinstance(obj,str): # Строки возвращаем как есть
            yield obj
        else: # Для остальных неитерируемыемых объектов
            yield obj
    try:
        result = flatten_gen(nested_obj)
        if gen:
            return result
        else:
             return list(result)
    except ValueError as e:
        raise ValueError(f"Ошибка при выравнивании структуры: {e}") from None

***Задание 6.3.2. Тестирование функции flatten_it***


In [104]:
normal_data = [1, {'a': [2, 3]}, (4, 5)]
try:
    result = flatten_it(normal_data)
    print( result) 
except ValueError as e:
    print("ошибка:", e)

[1, 'a', 2, 3, (4, 5)]


In [106]:
text = "hello"
try:
    result = flatten_it(text)
    print( result)
except ValueError as e:
    print("ошибка:", e)

['hello']


In [102]:
cyclic_dict = {'a': 1, 'b': 2}
cyclic_dict['self'] = cyclic_dict  # Добавляем ссылку на самого себя
try:
    result = list(flatten_it(cyclic_dict, gen=True))
    print( result)
except ValueError as e:
    print("ошибка:", e)  

ошибка: Ахтунг, обнаружен цикл
